In [6]:
from sota_list import LSTMNetwork
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

import torch
import codecs
import json
from pprint import pprint


In [ ]:
# Load the model weights
model_weights_path = "finetuned_saved_models/bilstm-bert-finetuned-segmented-extracted.pth"

# PML
plm_name = 'bert-finetuned-segmented'

In [ ]:
def load_llm_parts(model_name):

    # Load the config, model, and tokenizer
    config = AutoConfig.from_pretrained(model_name, output_hidden_states =True)

    return [
        AutoModelForSequenceClassification.from_pretrained(model_name, config=config),
        AutoTokenizer.from_pretrained(model_name)
    ]



# Initialize the model and load the weights
model = LSTMNetwork(768,128,5,True)
model.load_state_dict(torch.load(model_weights_path))
model.eval()

# Load the LLM fine-tuned model
llm_model, tokenizer = load_llm_parts(plm_name)

In [8]:
# Load the ERC datasets
def load_dataset(name, type):
    file_name = f'erc-datasets/{name}/{type}.json'
    with codecs.open(file_name, 'r', 'utf-8') as fr:
            return json.load(fr)

    return None

dataset = load_dataset('MELD','train')
pprint(dataset[0])

[{'emotion': 'neutral',
  'sentiment': '0',
  'speaker': 'Chandler',
  'utterance': "also I was the point person on my company's transition from "
               'the KL-5 to GR-6 system.'},
 {'emotion': 'neutral',
  'sentiment': '0',
  'speaker': 'The Interviewer',
  'utterance': "You must've had your hands full."},
 {'emotion': 'neutral',
  'sentiment': '0',
  'speaker': 'Chandler',
  'utterance': 'That I did. That I did.'},
 {'emotion': 'neutral',
  'sentiment': '0',
  'speaker': 'The Interviewer',
  'utterance': "So let's talk a little bit about your duties."},
 {'emotion': 'surprise',
  'sentiment': '1',
  'speaker': 'Chandler',
  'utterance': 'My duties?  All right.'},
 {'emotion': 'neutral',
  'sentiment': '0',
  'speaker': 'The Interviewer',
  'utterance': "Now you'll be heading a whole division, so you'll have a lot "
               'of duties.'},
 {'emotion': 'neutral',
  'sentiment': '0',
  'speaker': 'Chandler',
  'utterance': 'I see.'},
 {'emotion': 'neutral',
  'sentiment

In [ ]:
def perform_classification(model, llm_model, tokenizer, text):

    # Tokenize
    token_ids = tokenizer(
        text, 
        truncation = True, 
        return_tensors='pt', 
        max_length = 512, 
        add_special_tokens=True
    )

    # Extract CLS
    cls_output = llm_model(**token_ids)
    cls_output = cls_output.hidden_states[-1][0,0,:]

    # Get the output
    output = model(cls_output)

    # Convert to labels
    scores = torch.sigmoid(output)
    print(scores)

    # Return
    return None

In [22]:
for conversation in dataset:
    
    for utt_data in conversation:
        perform_classification(model, llm_model, tokenizer, utt_data['utterance'])
    break

AttributeError: 'list' object has no attribute 'dim'

In [ ]:
# Save the results